In [1]:
import os
import sys
import PATH
import numpy as np
import pandas as pd
import utils

import seaborn as sns
from matplotlib import pyplot as plt

from importlib import reload
import analysis_for_MTL as A_MTL
import ribo_public.parse_ribosome as Pribo
from sklearn import preprocessing

from models import Backbone
from models import reader
from models import train_val
from models import Cross_stitch
from models.popen import Auto_popen
from models.loss import Dynamic_Task_Priority,Dynamic_Weight_Averaging

/home/wergillius/Project/UTR_VAE


In [28]:
reload(train_val)

<module 'models.train_val' from '/home/wergillius/Project/UTR_VAE/models/train_val.py'>

In [2]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset,DataLoader,random_split


In [13]:
a5b5_config = "/home/wergillius/Project/UTR_VAE/log/CrossStitch/CrossStitch_Model_MTL/fast_distibution/a5b5_g2_11.ini"
a5b5_popen = Auto_popen(a5b5_config)

# logger = utils.setup_logs(a5b5_popen.vae_log_path)

In [4]:
NBT_dir = "/data/users/wergillius/UTR_VAE/Ex_data"
round_random_index = np.load(os.path.join(NBT_dir,'random_index.M.npy'))

# define backbone task model

In [5]:
recon_config = "/home/wergillius/Project/UTR_VAE/log/Backbone/Reconstruction_MTL/pretrain_with_um2_p1_design/kld1e_5_l1.ini"

motif_config = "/home/wergillius/Project/UTR_VAE/log/Backbone/Motif_detection_MTL/pretrain_with_unm2_p1_design/eight_motif_test.ini"

log_dir = "/home/wergillius/Project/UTR_VAE/log/Backbone/RL_regressor_MTL/fast_distribution"
slr_l1 = os.path.join(log_dir,"student_model.ini")

task_config = {
    'RL':slr_l1, 'Recons':recon_config, 'Motif':motif_config
}

In [6]:
task_popen = {}
backbone = {}
for task in a5b5_popen.tasks:
    task_popen[task] = Auto_popen(task_config[task])
    if task == 'RL':
        task_popen[task].vae_pth_path = "/data/users/wergillius/UTR_VAE/pth/RL_regressor_MTL/fast_distribution/student_round0-model_best.pth"
    task_model = task_popen[task].Model_Class(*task_popen[task].model_args)
    utils.load_model(task_popen[task],task_model,logger)
    backbone[task] = task_model

 	 	 ==============<<< encoder load from >>>============== 	 	 

 	 	 ==============<<< encoder load from >>>============== 	 	 

 	/data/users/wergillius/UTR_VAE/pth/RL_regressor_MTL/fast_distribution/student_round0-model_best.pth

 	/data/users/wergillius/UTR_VAE/pth/RL_regressor_MTL/fast_distribution/student_round0-model_best.pth

 	 	 ==============<<< encoder load from >>>============== 	 	 

 	 	 ==============<<< encoder load from >>>============== 	 	 

 	/data/users/wergillius/UTR_VAE/pth/Reconstruction_MTL/pretrain_with_um2_p1_design/kld1e_5_l1-model_best.pth

 	/data/users/wergillius/UTR_VAE/pth/Reconstruction_MTL/pretrain_with_um2_p1_design/kld1e_5_l1-model_best.pth

 	 	 ==============<<< encoder load from >>>============== 	 	 

 	 	 ==============<<< encoder load from >>>============== 	 	 

 	/data/users/wergillius/UTR_VAE/pth/Motif_detection_MTL/pretrain_with_unm2_p1_design/eight_motif_test-model_best.pth

 	/data/users/wergillius/UTR_VAE/pth/Motif_detection_MTL/pretra

In [15]:
left_model_args = a5b5_popen.model_args

## define dataset

In [7]:
dataset_func = lambda x : reader.MTL_enc_dataset(
    DF=x,pad_to=a5b5_popen.pad_to,aux_columns=a5b5_popen.aux_task_columns,
    input_col=a5b5_popen.other_input_columns)

class sub_round_dataset(Dataset):
    def __init__(self,entire_val_set,round_x__index):
        self.all_data = entire_val_set
        self.sampled_index = round_x__index
        
    def __len__(self):
        return len(self.sampled_index)
        
    def __getitem__(self,index):
        i = self.sampled_index[index]
        return self.all_data[i]

def get_two_set(dataset,ratio=a5b5_popen.train_test_ratio,seed = 42):
    total_len = len(dataset)
    lengths = [int(total_len*sub_ratio) for sub_ratio in ratio[:-1]]
    lengths.append(total_len-sum(lengths))         # make sure the sum of length is the total len

    train_set,val_set = random_split(dataset,lengths,generator=torch.Generator().manual_seed(seed))
    return train_set,val_set

def DF_to_dataloader(full_df,POPEN):
    df_ls = [full_df]

    dataset_func = lambda x : reader.MTL_enc_dataset(DF=x,pad_to=POPEN.pad_to,
                                aux_columns=POPEN.aux_task_columns,input_col=POPEN.other_input_columns)

    loader_ls = reader.get_splited_dataloader(dataset_func,df_ls,ratio=POPEN.train_test_ratio,
                                    batch_size=POPEN.batch_size,num_workers=4,seed=42) #
    return loader_ls

In [8]:
task_popen['RL'].train_test_ratio

[0.8, 0.2]

In [9]:
DF = pd.read_csv(task_popen['RL'].csv_path)
dataset = dataset_func(DF)
# the train_set and val_set were used to train the teacher model
train_set, val_set = get_two_set(dataset,task_popen['RL'].train_test_ratio)  

# val_df is the entire dataset used to train small_model
val_df = DF.iloc[val_set.indices]

/home/wergillius/.conda/envs/pytorch/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (28,37,82,83,84,85,86,87,88,89) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
round_df = val_df.iloc[round_random_index[0]]
round_train_loader,round_val_loader,_ = DF_to_dataloader(round_df,a5b5_popen)

In [11]:
a5b5_popen.vae_log_path

'/home/wergillius/Project/UTR_VAE/log/CrossStitch/CrossStitch_Model_MTL/fast_distibution/a5b5_g2_11.log'

In [17]:
round_x_df = pd.DataFrame([])

task_popen['RL'].vae_pth_path = "/data/users/wergillius/UTR_VAE/pth/RL_regressor_MTL/fast_distribution/student_round-1-model_best.pth"
a5b5_popen.vae_pth_path = '/data/users/wergillius/UTR_VAE/pth/CrossStitch_Model_MTL/fast_distibution/a5b5_round-1-model_best.pth'

for round_x in range(10):
    
    # incredent dataset 
    round_x_df = round_x_df.append(val_df.iloc[round_random_index[round_x]])
    round_x_trainloader,round_x_valloader,_ = DF_to_dataloader(round_x_df,a5b5_popen)
    
    # reload RL pretrain
    task_popen['RL'].vae_pth_path = task_popen['RL'].vae_pth_path.replace("_round%s"%(round_x-1),"_round%s"%round_x)
    task_model = task_popen['RL'].Model_Class(*task_popen['RL'].model_args)
    utils.load_model(task_popen['RL'],task_model,logger)
    backbone['RL'] = task_model
    
    # 
    a5b5_popen.model_args = [backbone] + left_model_args
    model = a5b5_popen.Model_Class(*a5b5_popen.model_args).cuda(a5b5_popen.cuda_id)
    
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                            lr=a5b5_popen.lr,
                            betas=(0.9, 0.98), 
                            eps=1e-09, 
                            weight_decay=a5b5_popen.l2)
    if a5b5_popen.loss_schema == 'DTP':
        a5b5_popen.loss_schedualer = Dynamic_Task_Priority(a5b5_popen.tasks,a5b5_popen.gamma,a5b5_popen.chimerla_weight)
    
    logger.info("####################################")
    logger.info("############# ROUND : %s  #########"%round_x)
    logger.info("####################################")
    
    for epoch in range(10):
        logger.info("\n===============================|    epoch {}   |===============================\n".format(epoch))
        train_val.train(dataloader=round_x_trainloader,
                        model=model,optimizer=optimizer,
                        popen=a5b5_popen,epoch=epoch)

    val_total_loss,val_avg_acc = train_val.validate(round_x_valloader,model,popen=a5b5_popen,epoch=epoch)

    a5b5_popen.vae_pth_path = a5b5_popen.vae_pth_path.replace("_round%s"%(round_x-1),"_round%s"%round_x)

    utils.snapshot(a5b5_popen.vae_pth_path, {
                            'epoch': epoch + 1,
                            'validation_acc': val_avg_acc,
                            'state_dict': model.state_dict(),
                            'validation_loss': val_total_loss,
                            'optimizer': optimizer.state_dict(),
                        })

 	 	 ==============<<< encoder load from >>>============== 	 	 

 	 	 ==============<<< encoder load from >>>============== 	 	 

 	/data/users/wergillius/UTR_VAE/pth/RL_regressor_MTL/fast_distribution/student_round0-model_best.pth

 	/data/users/wergillius/UTR_VAE/pth/RL_regressor_MTL/fast_distribution/student_round0-model_best.pth

####################################
####################################
############# ROUND : 0  #########
############# ROUND : 0  #########
####################################
####################################

===============================|    epoch 0   |===============================


===============================|    epoch 0   |===============================

    0 /   375 (0.0%):	 RL_loss:1.1231179	 Recons_loss:0.0832776	 Motif_loss:7.4672832	 Total:0.4019336	 RL_Acc:0.3125000	 Recons_Acc:0.6404703	 Motif_Acc:0.6601562	 DTP_wt_RL:0.2615663	 DTP_wt_Recons:0.6497658	 DTP_wt_Motif:0.0072386	 Mean_Total:0.4019336
    0 /   375 (0.0%):	 RL_lo